In [29]:
print(isinstance(int, object))
print(isinstance(int, type))
print(issubclass(int, object))
print(issubclass(int, type))
print(isinstance(object, type))
print(issubclass(object, type))
print(issubclass(type, object))
print(isinstance(type, object))

True
True
True
False
True
False
True
True


在Python的世界中，object是父子关系的顶端，所有的数据类型的父类都是它；type是类型实例关系的顶端，所有对象都是它的实例。obj.\_\_bases__可以查看父类

In [32]:
id(globals())

4406660480

In [33]:
id(locals())

4406660480

In [3]:
import sys
a=-6
b=-5
c=256
dd=257
ee=258
print(sys.getrefcount(a))
print(sys.getrefcount(b))
print(sys.getrefcount(c))
print(sys.getrefcount(dd))
print(sys.getrefcount(ee))
e=int(1)
import copy
f = copy.deepcopy(e)
print(sys.getrefcount(e))
print(sys.getrefcount(f))

2
11
99
3
3
3195
3195


-5至256的数做了缓存，为什么257比-6多一个引用计数？但在ipython中测试并无此问题

In [63]:
a,b=1,3
eval('a + b')

4

In [65]:
g = {'a':10,'b':5}
l = {'b':20}
eval('a+b', g, l)

30

eval和exec默认使用当前的名字空间，可以传入名字空间

In [83]:
x = 1<<1000
print(x)
sys.getsizeof(x)

10715086071862673209484250490600018105614048117055336074437503883703510511249361224931983788156958581275946729175531468251871452856923140435984577574698574803934567774824230985421074605062371141877954182153046474983581941267398767559165543946077062914571196477686542167660429831652624386837205668069376


160

In [84]:
6&11

2

In [92]:
'ashakklskl' is sys.intern('ashakklskl')

True

In [97]:
a = 'helloworld!'
b = 'helloworld!'
print(a is b)
print(a is sys.intern(a))
print(a is sys.intern('helloworld!'))

False
True
True


池化：python中的变量实际上是名字，也是字符串实例，在不同的名字空间中如何去共享这些实例以节约内存，就是用一个字符串池来实现。

In [103]:
b = bytearray(b'abc')
d = b'abc'
print(len(b))
print(len(d))
print(d)
print(b+d)

3
3
b'abc'
bytearray(b'abcabc')


bytes是一次性内存分配的，bytearray可以按需扩张也可提前分配。

In [196]:
e = bytearray(b'abcdefg')
f = e[2:5]
print(f)
e[3] = 0x77
print(f)
print(e)
v = memoryview(e)
x = v[2:5]
x[0] = 0x77
print(v)
print(e)

bytearray(b'cde')
bytearray(b'cde')
bytearray(b'abcwefg')
bytearray(b'abwwefg')


e的改变没有影响f，说明切片是字节数据的复制，使用内存视图可以修改到原数据。
内存视图是对目标内存进行引用，它需要目标对象支持缓冲协议，能否修改要看原对象是否支持。

- 列表的内部结构由两部分组成，保存元素数量和内存分配计数的头部，以及存储元素指针的独立数组。
- 固定长度的数组头部结构是为实现内存复用。创建时，优先从复用区获取。被回收时若未超出复用数量限制（80），会被放回复用区而不是交还内存。每次真正需要分配和释放的是元素指针数组。
- 元组是不可变类型，所以它的指针数组可以一次性玩成内存分配。系统会缓存复用一定长度的元组内存（含指针数组）。创建时，按所需长度提取复用，没有额外的内存分配。
- 数组和列表、元组的本质区别在于：元素单一类型和内容嵌入。由于可指定更紧凑的数字类型，数组可以节约更多的内存。内容嵌入也避免了对象的额外开销，减少了活跃对象的数量和内存分配的次数。

Python3.6字典实现：
- keys.entries和values用数组按添加顺序存储主键和值引用。实际哈希表由keys.indices数组承担，通过计算主键哈希值查找合适的位置，然后在此存储主键在keys.entries的实际索引。
- 系统分别缓存复用80个dict和keys，其中包括长度为8的entries内存。对于大量小字典而言，直接使用无须任何内存分配操作。回收时，凡内存被扩张过的都被放弃。
- 从开放寻址法的实现方式看，它不适合处理大数据。

In [11]:
# 几种特殊的构建方式
kvs = [['a', 1], ('b', 3)]
c = dict(kvs)
print(c)
d = dict(c, e=9)
print(d)
x = dict.fromkeys(d, 0)
print(x)
y = dict.fromkeys(['h','i'], 1)
z = dict.fromkeys('j', 2)
print(y, z)
# 如果有a，返回a的值，否则设置a
x.setdefault('a', 10)
y.setdefault('a', 10)
print(x, y)

{'a': 1, 'b': 3}
{'a': 1, 'b': 3, 'e': 9}
{'a': 0, 'b': 0, 'e': 0}
{'h': 1, 'i': 1} {'j': 2}
{'a': 0, 'b': 0, 'e': 0} {'h': 1, 'i': 1, 'a': 10}


字典没有独立的只读版本，无论传引用还是复制品都有不足，直接引用有被接收方修改内容的风险，复制品无法获知字典的变化。视图能同步读取字典的内容，却无法修改。且可选择不同粒度的内容进行传递，如此可将接收方限定为指定模式下的观察员。

In [16]:
# 视图支持集合运算，弥补字典功能不足。
kc = c.keys()
ky = y.keys()
print(kc & ky) # 交集
print(kc | ky) # 并集
print(kc - ky) # 差集，仅属于a，不属于b
print(kc ^ ky) # 对称差集，仅属于a 加 仅属于b， 等于并集减交集
ks = kc&ky
c.update({k:y[k] for k in ks}) # 利用交集结果提取待更新内容
print(c)

{'a'}
{'h', 'a', 'b', 'i'}
{'b'}
{'h', 'b', 'i'}
{'a': 10, 'b': 3}


集合：
- 判重公式： (a is b) OR ( hash(a) == hash(b) AND a == b)
- set 和 frozenset 内部实现相同，用数组实现的哈希表存储元素对象引用，这就要求元素必须是可哈希类型
- 集合对象自带一个长度为8的小数组（small table），这对多数简单集合运算有益，可避免额外的内存分配。只有超出容量限制，才分配更大的数组内存（entry table）
- 集合使用频率不及列表、字典，其内部没有采用缓存复用的策略。实现方式也决定了是无序存储，标准库也没有有序实现


In [17]:
ks.remove('f') # 引发异常
ks.discard('f') # 不引发异常

KeyError: 'f'

In [23]:
class User:
    def __init__(self, uid):
        self.uid = uid
        
u1 = User(1)
u2 = User(1)
s = set((u1, u2))
print(s)

{<__main__.User object at 0x103b94cc0>, <__main__.User object at 0x103b94c50>}


自定义对象的哈希实现是返回随机值，\_\_eq__仅比较自身，所以无法通过set去重，需要重载这两个方法。另外如果只重载eq方法，会发现对象是不可哈希的。

In [31]:
class User:
    def __init__(self, uid, name):
        self.uid = uid
        self.name = name
        
    def __hash__(self):
        return hash(self.uid)
    
    def __eq__(self, other):
        return self.uid == other.uid
        
u1 = User(1, 'ss')
u2 = User(1, 'bb')
s = set((u1, u2))
print(s)

{<__main__.User object at 0x103babeb8>}
